<a href="https://it-omscholing.nl/locaties/hogeschool-rotterdam/">
<div>
<a><img src='pics/banner.PNG'/></a>
</div>
<div>
<a href=''><img src='pics/miw.PNG'/></a>
<em>Author: Jeroen Boogaard</em>
</div>
</a>

<h1>Practicum Computer Vision</h1>

<h2>Imports<h2>

In [ ]:
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from sklearn.model_selection import train_test_split
import random
import requests
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import datasets, layers, models
import zipfile

**Gebruik een <i>environment variable</i> voor het instellen van het logleven van TensorFlow**

In [ ]:
os.environ['TF_CPP_MIN_LEVEL'] = '2'

<h2>Data Collection</h2>

<h3>Data downloaden</h3>

In [ ]:
dataUrl = requests.get("http://madm.dfki.de/files/sentinel/EuroSAT.zip") # foto's verdeeld over 10 klassen.
dataFile = zipfile.ZipFile(io.BytesIO(dataUrl.content))

In [ ]:
# get file names
txtFiles = list() # lijst wordt aangemaakt
# print(txtFiles)

for textFile in dataFile.namelist():
    txtFiles.append(textFile)
    # print(txtFile)

# print(txtFiles)
# print(textFile) # zal de laaste geschreven zijn. in dit geval crop_736 
# print(txtFile)
# print(txtFiles[6777])

<h3>Data filteren</h3>

In [ ]:
txtFiles = [x for x in txtFiles if ".jpg" in x] # manier op de .jpg te scheiden van de andere bestandtypes. 


<h3>Data inlezen</h3>

In [ ]:
imageObjects = np.zeros([len(txtFiles), 64, 64, 3]) 
# aanmaken van lege array van 64x64x3 pixels/datatype. (aantal arrays is 'len' van 'tekstfiles'.)

i = 0

for pic in txtFiles:
  imageObjects[i] = np.asarray(Image.open(dataFile.open(pic))).astype('uint8')/255
  i += 1
# hier wordt de afbeelding (genormaliseerd) in de array gezet. Uitkomst -> afbeelding in de matrixvorm genormaliseerd tussen 0 en 1. 

print(imageObjects[0])    

**Gebruik de bestandsnamen voor het verkrijgen van de labels**

In [ ]:
imageLabels = np.empty(len(txtFiles), dtype = 'S20')
# aanmaken van lege arrays. Net zoveel als de 'len' van txtFiles'

i = 0

for label in txtFiles:    #label wordt gecreeerd '/' te splitten van txtFiles.
    txtFiles[i] = label.split('/')[1]
    i += 1

print(label[1])
print(txtFiles[7000]) 


In [ ]:
# change them to integers in alphabetical order
labelNames, labelNumbers = np.unique(txtFiles, return_inverse=True)
labelDict = dict(zip(np.unique(labelNumbers), labelNames))
print(labelNames)
print(labelNumbers)

np.array(np.unique(labelNumbers, return_counts=True)).T

<h3>Data splitten</h3>

In [ ]:
trainSet, testSet, trainLabels, testLabels = train_test_split(imageObjects, labelNumbers, stratify = labelNumbers, train_size = 0.75, random_state=42)
print(trainSet.shape)
nrOfImages = len(trainSet)

<h3>Data inspecteren</h3>

In [ ]:
def inspectData():
    
    plt.figure(figsize=(10,10))
    
    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        number = i
        plt.imshow(trainSet[number])
        plt.xlabel(labelNames[trainLabels[number]]) 
    
    plt.show()

In [ ]:
inspectData()    

<h2>Maak een model</h2>

<h3>Definieer de netwerktopologie</h3>

In [ ]:
class_names = labelNames

In [ ]:
num_classes = len(labelNames)

model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes))

model.summary()




<h3>Exercise 1</h3>
<p>
    Vul de netwerktopologie tot een bruikbaar Neural Netwerk onstaat.
</p>

<h3>Definieer de trainparameters</h3>

In [ ]:
lossFunction = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gdAlgorithm = keras.optimizers.Adam(learning_rate=0.001)
nrOfEpochs = 5

<h3>Train het model</h3>

In [ ]:
model.compile(optimizer=gdAlgorithm, loss=lossFunction, metrics="accuracy")
history = model.fit(trainSet, trainLabels, epochs=nrOfEpochs, batch_size=64, verbose=2)


In [ ]:
# model.save('/Users/boyfrankclaesen/workspace/makeAIWork2/opdrachten/practica/week07/cnn_trained')

<h3>Test het model</h3>

<h3>Exercise 2</h3>
<p>
    Test het Neural Netwerk met de testset en laat zien of het in staat is de juiste class te vinden.
</p>

In [ ]:
# validationPicture_path = ('makeAIWork2/opdrachten/practica/week07/Highway_1886.jpg')


In [ ]:
valPictureArray = (imageObjects[1886])

print(valPictureArray)

